# Hyperparameter search across optimizer, learning rate, weight decay

In [1]:
# Global imports
import os
import json
import sys
sys.path.insert(0, 'src')

In [2]:
# Local imports
from utils.util import read_json, informal_log
from train import main as train_fn
from test import main as test_fn
from parse_config import ConfigParser
from data_loader import data_loaders

In [6]:
# architectures = [
#     "vgg11_bn",
#     "vgg13_bn",
#     "vgg16_bn",
#     "vgg19_bn",
#     "resnet18",
#     "resnet34",
#     "resnet50",
#     "densenet121",
#     "densenet161",
#     "densenet169",
#     "mobilenet_v2",
#     "googlenet",
#     "inception_v3"]

architectures = ["resnet18"]
optimizers = ["Adam", "SGD"]
learning_rates_dict = {
    "Adam": [1e-3, 1e-4, 1e-5, 1e-6],
    "SGD": [0.1, 0.01, 1e-3, 1e-4]
}
weight_decays = [0, 1e-1, 1e-2, 1e-3, 1e-4]
n_epochs = 100

config_path = 'configs/cinic10_imagenet_train_hyperparameter_search.json'
config_dict = read_json(config_path)

# Test values

# architectures = ["vgg11_bn"]
# optimizers = ["Adam", "SGD"]
# learning_rates = [0.1, 0.01]
# weight_decays = [0, 0.1]
# n_epochs = 15

In [5]:
# Initialize data loaders
data_loader_args = dict(config_dict["data_loader"]["args"])
train_data_loader = data_loaders.CINIC10DataLoader(
    **data_loader_args,
    shuffle=True,
    split='train')
val_data_loader = data_loaders.CINIC10DataLoader(
    **data_loader_args,
    shuffle=False,
    split='valid')
test_data_loader = data_loaders.CINIC10DataLoader(
    **data_loader_args,
    shuffle=False,
    split='test')

In [ ]:
# print what we're searching over
print("Architectures: {}".format(architectures))
print("Optimizers: {}".format(optimizers))
print("Learning Rates: {}".format(learning_rates_dict.items()))
print("Weight Decays: {}".format(weight_decays))
print("N_epochs: {}".format(n_epochs))

for architecture in architectures:
     # Read in config file fresh
    config_dict = read_json(config_path)
    
    config_dict.update({"name": "HyperParameterSearch_{}".format(architecture)})
    summary_file_path = os.path.join(
        config_dict["trainer"]["save_dir"], 
        config_dict["name"], 
        'summary.txt')

    informal_log("Train data path: {}".format(train_data_loader.get_data_dir()))
    informal_log("Validation data path: {}".format(val_data_loader.get_data_dir()))
    informal_log("Test data path: {}".format(test_data_loader.get_data_dir()))

    informal_log("Test results for {}...".format(architecture), summary_file_path)
    
    # Set number of epochs
    config_dict["trainer"].update({"epochs": n_epochs})
    
    # Update model architecture information
    for key, value in config_dict["arch"]["args"].items():
        config_dict["arch"]["args"].update({key: value.format(architecture)})

    # Hyperparameter search
    for optimizer in optimizers:
        config_dict["optimizer"].update({"type": optimizer})
        config_dict["optimizer"]["args"].pop("amsgrad", None)
        learning_rates = learning_rates_dict[optimizer]
        for learning_rate in learning_rates:
            config_dict["optimizer"]["args"].update({"lr": learning_rate})
            for weight_decay in weight_decays:
                # Update configuration dictionary
                config_dict["optimizer"]["args"].update({"weight_decay": weight_decay})
                
                # Set name of run ID based on hyperparameters
                run_id = "optim_{}-lr_{}-wd_{}".format(
                    config_dict["optimizer"]["type"],
                    config_dict["optimizer"]["args"]["lr"],
                    config_dict["optimizer"]["args"]["weight_decay"]
                )
                
                # Log parameters in summary
                informal_log("Optimizer: {} \tLearning Rate: {} \tWeight Decay: {}".format(
                    optimizer, learning_rate, weight_decay
                ), summary_file_path)
                informal_log("N_epochs: {}".format(n_epochs), summary_file_path)
                
                # Create config object and train
                try:
                    config = ConfigParser(config_dict, run_id=run_id)
                except ValueError as e:
                    print(e)
                    print("Skipping hyperparameter configuration")
                    continue
                train_fn(config,
                    train_data_loader=train_data_loader,
                    val_data_loader=val_data_loader)
                
                # Obtain best model and run on test set
                best_ckpt_path = os.path.join(config.save_dir, 'model_best.pth')
                informal_log("Best checkpoint at {}".format(best_ckpt_path), summary_file_path)
                test_run_id = os.path.join(run_id, 'test')
                config_test = ConfigParser(config_dict, run_id=test_run_id, resume=best_ckpt_path)
                test_results = test_fn(config_test, test_data_loader=test_data_loader)
                
                informal_log("Test results:\n{}".format(test_results), summary_file_path)
                informal_log("\n---***---\n", summary_file_path)
'''
for each arch:
    change config name=HyperParamSearch_{arch.type}, arch.type, arch.checkpoint_path,
    for each optimizer:
        set config optimizer.type
        for each lr:
            set config optimizer.args.lr
            for each weight decay:
                setconfig.optimizer.weight_decay
                config.run_id = {dataset}_{arch.type}_{optim}_{lr}_{weight_decay}
                config = ConfigParser(config)
                train.main(config)
                

'''


Architectures: ['resnet18']
Optimizers: ['Adam', 'SGD']
Learning Rates: dict_items([('Adam', [0.001, 0.0001, 1e-05, 1e-06]), ('SGD', [0.1, 0.01, 0.001, 0.0001])])
Weight Decays: [0, 0.1, 0.01, 0.001, 0.0001]
N_epochs: 100
Train data path: data/cinic-10-imagenet/train
Validation data path: data/cinic-10-imagenet/valid
Test data path: data/cinic-10-imagenet/test
Test results for resnet18...
Optimizer: Adam 	Learning Rate: 0.001 	Weight Decay: 0
N_epochs: 100
Created CIFAR10PretrainedModel model with 11173962 trainable parameters
Restored weights from external_code/PyTorch_CIFAR10/cifar10_models/state_dicts/resnet18.pt
Checkpoint save directory: saved/HyperParameterSearch_resnet18/optim_Adam-lr_0.001-wd_0/models


2022-11-21 09:26:38.725924: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-21 09:26:38.991092: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-21 09:26:42.048045: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /n/fs/ac-project/anaconda3/lib:
2022-11-21 09:26:42.048174: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7

Train Epoch: 1 [0/70000 (0%)] Loss: 1.819061
Train Epoch: 1 [12288/70000 (18%)] Loss: 2.063884
Train Epoch: 1 [24576/70000 (35%)] Loss: 2.021370
Train Epoch: 1 [36864/70000 (53%)] Loss: 2.011982
Train Epoch: 1 [49152/70000 (70%)] Loss: 2.008869
Train Epoch: 1 [61440/70000 (88%)] Loss: 1.904796
    epoch          : 1
    loss           : 2.0214212288821702
    accuracy       : 0.43487682481751827
    val_loss       : 2.0208689030939646
    val_accuracy   : 0.4352454542492179
Saving checkpoint from epoch 1 to saved/HyperParameterSearch_resnet18/optim_Adam-lr_0.001-wd_0/models/model_best.pth ...
Train Epoch: 2 [0/70000 (0%)] Loss: 1.962234
Train Epoch: 2 [12288/70000 (18%)] Loss: 1.985071
Train Epoch: 2 [24576/70000 (35%)] Loss: 1.952466
Train Epoch: 2 [36864/70000 (53%)] Loss: 1.928913
Train Epoch: 2 [49152/70000 (70%)] Loss: 1.933032
Train Epoch: 2 [61440/70000 (88%)] Loss: 1.907607
    epoch          : 2
    loss           : 1.930099165787662
    accuracy       : 0.5281767303180397
   

In [9]:
# print stuff
print(config["optimizer"])
best_ckpt_path = os.path.join(config.save_dir, 'model_best.pth')
print(best_ckpt_path)

OrderedDict([('type', 'Adam'), ('args', OrderedDict([('lr', 0.1), ('weight_decay', 0), ('amsgrad', False)]))])
saved/HyperParameterSearch_vgg11_bn/optim_Adam-lr_0.1-wd_0/models/model_best.pth
